<a href="https://colab.research.google.com/github/Thaleslsilva/DataScience/blob/master/Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de Caso - Previsão de Palavras com Base no Contexto e Visualização com PCA

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Instala o PyTorch
!pip install -q torch 

In [ ]:
# Pacote para gráficos com Scikit-learn
!pip install -q scikit-plot

In [ ]:
# Imports
import torch
import scipy
import sklearn
import scikitplot
import numpy as np
import torch.nn.functional as F
from torch.optim import SGD
from torch.autograd import Variable, profiler
from sklearn.decomposition import PCA
from scipy.spatial.distance import cosine
from scikitplot.decomposition import plot_pca_2d_projection
%matplotlib inline

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Thales de Lima Silva" --iversions

### Preparação dos Dados

In [ ]:
# Corpus
corpus = ['ele é um rei',
          'ela é uma rainha',
          'ele é um homem',
          'ela é uma mulher',
          'Madrid é a capital da Espanha',
          'Berlim é a capital da Alemanha',
          'Lisboa é a capital de Portugal']

In [ ]:
# Construindo o vocabulário com tokenização
palavras = []
for sentence in corpus:
    for palavra in sentence.split():
         if palavra not in palavras:
            palavras.append(palavra)

In [ ]:
# Visualiza os dados
palavras

In [ ]:
# Criamos o mapeamento palavra - índice   
word2idx = {w:idx for (idx, w) in enumerate(palavras)}
word2idx

In [ ]:
# Criamos o mapeamento inverso índice - palavra
idx2word = {idx:w for (idx, w) in enumerate(palavras)}
idx2word

In [ ]:
# Tamanho do vocabulário
tamanho_vocab = len(word2idx)
tamanho_vocab

### Construção do Modelo

In [ ]:
# Função para gerar os embeddings
def get_word_embedding(word):
    word_vec_one_hot = np.zeros(tamanho_vocab)
    word_vec_one_hot[word2idx[word]] = 1
    return word_vec_one_hot

In [ ]:
# Função para gerar os vetores, da palavra central e do contexto
def gera_vetores():
    for sentence in corpus:
        words = sentence.split()
        indices = [word2idx[w] for w in words]
        
        # Loop pelo range de índices
        # Aqui geramos o vetor da palavra central em i
        # E geramos o vetor de contexto
        for i in range(len(indices)):
            for w in range(-window_size, window_size + 1):
                context_idx = i + w
                if context_idx < 0 or context_idx >= len(indices) or i == context_idx:
                    continue
                    
                # Gera os vetores    
                center_vec_one_hot = np.zeros(tamanho_vocab)
                center_vec_one_hot[indices[i]] = 1
                context_idx = indices[context_idx]
                                
                yield center_vec_one_hot, context_idx

In [ ]:
# Hiperparâmetros
embedding_dims = 10
window_size = 2

Definição dos pesos da rede neural.

- W1 é uma matriz de pesos de dimensões embedding_dims x tamanho_vocab
- W2 é uma matriz de pesos de dimensões tamanho_vocab x embedding_dims

Os pesos (ou coeficientes ou parâmetros) é aquilo que a rede aprende durante o treinamento. Como no início não sabemos qual o valor ideal de pesos (isso é o que queremos descobrir) iniciamos com valores randômicos usando torch.randn().

Ao final do aprendizado, o modelo em si nada mais é do que os valores ideais de W1 e W2.

In [ ]:
# Definição dos pesos da rede neural
W1 = Variable(torch.randn(embedding_dims, tamanho_vocab).float(), requires_grad = True)
W2 = Variable(torch.randn(tamanho_vocab, embedding_dims).float(), requires_grad = True)

In [ ]:
# Treinamento
print("\nIniciando o Treinamento...\n")
for epoch in range(1001):
    
    # Inicializa o erro médio da rede
    avg_loss = 0
    
    # Inicializa o controle do número de amostras
    samples = 0
    
    # Loop pelos dados (vetores de entrada)
    for data, target in gera_vetores():
        
        # Coleta x (vetor da palavra central)
        x = Variable(torch.from_numpy(data)).float()
        
        # Coleta y (vetor do contexto)
        y_true = Variable(torch.from_numpy(np.array([target])).long())
        
        # Atualiza o número de amostras
        samples += len(y_true)
        
        # Resultado da multiplicação entre os pesos e as primeiras camadas da rede
        a1 = torch.matmul(W1, x)
        a2 = torch.matmul(W2, a1)

        # A função softmax entrega a probabilidade da previsão da rede
        log_softmax = F.log_softmax(a2, dim = 0)

        # Previsão da rede
        network_pred_dist = F.softmax(log_softmax, dim = 0)
        
        # Calcula o erro, comparando a previsão da rede com o valor real 
        # (como fazemos em qualquer modelo de aprendizagem supervisionada)
        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        
        # Erro médio
        avg_loss += loss.item()
        
        # Inicia o backpropagation
        loss.backward()

        # Atualiza o valor dos pesos para a próxima passada
        W1.data -= 0.002 * W1.grad.data
        W2.data -= 0.002 * W2.grad.data

        # Zera o valor do gradiente depois de atualizar os pesos
        W1.grad.data.zero_()
        W2.grad.data.zero_()
        
    # Imprime o erro da rede
    if epoch % 10 == 0:
        print('Erro de Treinamento:', avg_loss / samples)

print("\nTreinamento Concluído.")

### Teste do Modelo e Redução de Dimensionalidade com PCA

Para testar o modelo, tudo que precisamos é dos pesos, em nosso exemplo W1 e W2. Mas visualizar os dados é desafiador, pois a dimensionalidade é alta e quanto maior o número de palavras do vocabulário, mais complicado.

Uma alternativa, é reduzir a diemensionalidade dos dados. Convertemos todos os atributos em 2 componentes principais usando PCA (Principal Component Analysis) e com 2 componentes podemos visualizar os dados.

Cada componentes principal nada mais é do que a junção matemática da informação em outras variáveis. O PCA é um algoritmo de Machine Learning por si mesmo, da categoria de aprendizagem não supervisionada.

Vamos aplicar o PCA para visualizar os dados.

In [ ]:
# Cria o objeto para redução de dimensionalidade
pca = PCA(n_components = 2)

In [ ]:
# Treina o modelo PCA
pca.fit(W1.data.numpy().T)

In [ ]:
# Calcula a projeção PCA para o Plot
proj = pca.transform(W1.data.numpy().T)

In [ ]:
# Plot
ax = plot_pca_2d_projection(pca, 
                            W1.data.numpy().T, 
                            np.array(palavras), 
                            feature_labels = palavras, 
                            figsize = (16,10), 
                            text_fontsize = 12)

# Legenda
for i, txt in enumerate(palavras):
    ax.annotate(txt, (proj[i,0], proj[i,1]), size = 15)

Observe a legenda no gráfico acima! Palavras similares com base no contexto, estão com a "bolinha" com cores parecidas. No topo da lista temos países e cidades, depois pronomes e a palavra "capital", temos então homem, mulher, rainha e rei e por fim artigos e um verbo.

Tudo isso foi aprendido pela rede com base no contexto, que nada mais é do que a distância de cosseno entre as embeddings, os vetores que representam as palavras.

A visualização acima mostra que palavras que estão na mesma direção possui alguma similaridade, por exemplo "Alemanha" e "Berlim". Passe uma linha reta imaginária que "corta" as palavras "Alemanha" e "Berlim". Consegue? Se a resposta for sim, as palavras são similares. Abaixo terá outro exemplo.

Vamos extrair as distâncias com base na pergunta:

**Espanha está para Madrid, assim como Alemanha está para ?**

Vamos perguntar ao modelo.

In [ ]:
# Função para obter um vetor de palavras no peso W1 (esse é o contexto)
def get_word_vector_v(word):
    return W1[:, word2idx[word]].data.numpy()

In [ ]:
# Função para obter um vetor de palavras no peso W2 (essa é a palavra central)
def get_word_vector_u(word):
    return W2[word2idx[word],:].data.numpy()

In [ ]:
# Vamos obter os vetores das palavras
espanha = 1 * get_word_vector_v('Espanha') + 1 * get_word_vector_u('Espanha')
alemanha = 1 * get_word_vector_v('Alemanha') + 1 * get_word_vector_u('Alemanha') 
madrid = 1 * get_word_vector_v('Madrid') + 1 * get_word_vector_u('Madrid') 

In [ ]:
# Resultado
resultado = madrid - espanha + alemanha

In [ ]:
# Este é o resultado, ou seja, uma embedding que representa a palavra mais similar à palavra "Alemanha",
# com base na similaridade (contexto) entre "Polônia" e "Varsóvia".
resultado

In [ ]:
# Vamos extrair as distâncias de todas as outras palavras para a nossa palavra "secreta" que está 
# no vetor embedding chamado "resultado"
# Usamos a função cosine() do SciPy para calcular as distâncias
distancias = [(v, cosine(resultado, 1 * get_word_vector_u(v) + 1 * get_word_vector_v(v))) for v in palavras]

In [ ]:
# Visualiza as distâncias
distancias

Acima temos uma lista de tuplas com as distâncias de cada palavra para nosso "resultado". Vamos ordenar isso.

In [ ]:
# Ordenando a lista de tuplas pelo segundo elemento da tupla
distancias.sort(key = lambda tup: tup[1])  

In [ ]:
# Agora sim
distancias

O vetor "resultado" foi uma previsão do nosso modelo e as palavras "Madrid" e "Berlim" são as mais similares. Observe que "Berlim" é a palavra mais similar com base no conexto, uma vez que Madrid já foi usada em nossa fórmula.

Imagine que um vetor (uma flecha) sai da origem do sistema de coordenadas (Honestidade = 0 e Experiência = 0, chamaremos de ponto O) e termina no ponto X. Este vetor é usado para localizar o ponto no nosso espaço de características. Não é diferente de simplesmente dizer que X possui H = 0.4 e E = 0.2, é apenas outra maneira de ver isso.

**Em que contexto aparece a palavra Lisboa?**

Aqui é como se estivéssemos usando o modelo para previsão.

In [ ]:
# Extrai o contexto
context_to_predict = get_word_vector_v('Lisboa')

# Variável com o contexto a prever
hidden = Variable(torch.from_numpy(context_to_predict)).float()

# Executa o modelo e extrai as probabilidades 
# (executar o modelo nada mais é do que multiplicar os novos dados de entrada pelos pesos aprendidos no treinamento)
a = torch.matmul(W2, hidden)
probs = F.softmax(a, dim = 0).data.numpy()

# Imprime o resultado
for context, prob in zip(palavras, probs):
    print(f'{context}: {prob}')

O contexto da palavra "Lisboa" é representado pelas palavras "é", "a", "Portugal".

Nosso modelo não conseguiu aprender o contexto "capital". Quem sabe você consegue otimizar o treinamento do modelo e aumentar sua precisão.

# Fim